In [1]:
import keras
import tensorflow as tf
import keras.backend as K
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D, Activation, BatchNormalization, Input, Reshape, Dense, GlobalAveragePooling2D, Flatten
from keras.datasets import mnist, cifar10
from PIL import Image
from keras.utils import np_utils
from keras.utils import plot_model
from keras.optimizers import SGD
import pydot
import graphviz

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        '/data/cifar/train/',
        target_size=(32,32),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/data/cifar/test/',
        target_size=(32,32),
        batch_size=32,
        class_mode='categorical')

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [4]:
def class_model_head():
    model = Sequential()
    model.add(Convolution2D(16, (3,3), activation='relu', padding='same', input_shape=(32,32,3)))
    model.add(BatchNormalization())
    
    model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    return model

In [5]:
def class_model_tail():
    model = Sequential()
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same', input_shape=(15, 15, 128)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

    model.add(Convolution2D(10, (1,1), activation='relu'))
    model.add(AveragePooling2D(pool_size=(int(model.output.shape[1]), int(model.output.shape[2]))))
    model.add(Reshape((10,)))
    model.add(Activation('softmax'))
    return model

In [6]:
def disc_model():
    model = Sequential()
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same', input_shape=(15, 15, 128)))
    #model.add(BatchNormalization())
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
    #model.add(BatchNormalization())
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
    #model.add(BatchNormalization())
    model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
    #model.add(BatchNormalization())
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [7]:
def attach_head_to_tail(head, tail):
    model = Sequential()
    model.add(head)
    model.add(tail)
    return model

In [8]:
def combined_model(disc, feature_out):
    z = disc(feature_out)
    return z

In [9]:
def make_trainable(model, val):
    model.trainable = val
    for l in model.layers:
        l.trainable = val

In [10]:
class_head = class_model_head()
class_tail = class_model_tail()
class_combined = attach_head_to_tail(class_head, class_tail)
disc_tail = disc_model()
disc_combined = attach_head_to_tail(class_head, disc_tail)
disc_update = attach_head_to_tail(class_head, disc_tail)

# compile class combined for updating training accuracy
make_trainable(class_head, True)
make_trainable(class_tail, True)
class_combined.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# compile discriminator combined for updating discrimination
make_trainable(class_head, False)
make_trainable(disc_tail, True)
disc_combined.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# compile discriminator chain that allows head to update
make_trainable(class_head, True)
make_trainable(disc_tail, False)
disc_update.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
# only eat one gpu so i can work on other stuff
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [12]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
K.set_session(sess)

In [13]:
# i guess lets start with a trained classifier
class_combined.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=100)

Epoch 1/10
100/100 [==============================] - 3s - loss: 1.9687 - acc: 0.2875 - val_loss: 3.6325 - val_acc: 0.1062
Epoch 2/10
100/100 [==============================] - 2s - loss: 1.6740 - acc: 0.3956 - val_loss: 4.0681 - val_acc: 0.1116
Epoch 3/10
100/100 [==============================] - 2s - loss: 1.5741 - acc: 0.4275 - val_loss: 3.2819 - val_acc: 0.1659
Epoch 4/10
100/100 [==============================] - 2s - loss: 1.4337 - acc: 0.4906 - val_loss: 2.5368 - val_acc: 0.2169
Epoch 5/10
100/100 [==============================] - 2s - loss: 1.3601 - acc: 0.5209 - val_loss: 1.6462 - val_acc: 0.4169
Epoch 6/10
100/100 [==============================] - 2s - loss: 1.2821 - acc: 0.5503 - val_loss: 1.4322 - val_acc: 0.4878
Epoch 7/10
100/100 [==============================] - 2s - loss: 1.2687 - acc: 0.5628 - val_loss: 1.5055 - val_acc: 0.4966
Epoch 8/10
100/100 [==============================] - 2s - loss: 1.2218 - acc: 0.5728 - val_loss: 1.4078 - val_acc: 0.5197
Epoch 9/10
100/1

In [13]:
(X_test, Y_test) = validation_generator.next()
score = class_combined.evaluate(X_test, Y_test, verbose=0)
print(score)

[1.2548007965087891, 0.65625]


In [14]:
#sess=K.get_session()
#print(class_combined.weights[0].shape)
#sess.run(class_combined.weights[0])

In [15]:
for epoch in range(2):
    print("Epoch: ", epoch)
    #for batch in range(train_generator.samples/train_generator.batch_size):
    for batch in range(1000):
        # train the discriminator a little
        # grab a batch of train data
        (train_x, train_y) = train_generator.next()
        (val_x, val_y) = validation_generator.next()
        #train_x = np.ones((16, 15, 15, 128))
        # grab a batch of valid data
        #val_x = np.zeros(train_x.shape)
        # concatenate the two x values to provide mixed input to discriminator
        combined_x = np.concatenate((train_x, val_x))
        # create the labels for combined data
        combined_y = [1] * train_x.shape[0] + [0] * val_x.shape[0]
        # train the discriminator on this batch
        disc_combined.train_on_batch(combined_x, combined_y)
        # next, train the classification top based on the discriminator
        combined_y_reverse = [0] * train_x.shape[0] + [1]*val_x.shape[0]
        disc_update.train_on_batch(combined_x, combined_y_reverse)
        # finally, just train the classification a little bit
        class_combined.train_on_batch(train_x, train_y)
        #disc_combined.fit(combined_x, combined_y, epochs=1)
        #disc_combined.fit(combined_x, combined_y, epochs=1)       
        
    

('Epoch: ', 0)


KeyboardInterrupt: 

In [ ]:
#print(class_combined.weights[0].shape)
#sess.run(class_combined.weights[0])

In [ ]:
(train_x, train_y) = validation_generator.next()
print(np.mean(disc_combined.predict(train_x)))
(train_x, train_y) = train_generator.next()
print(np.mean(disc_combined.predict(train_x)))